# Импорт библиотек

In [ ]:
# !pip install lightgbm --install-option=--gpu
# !pip install catboost
# !pip install xgboost
# !pip install optuna

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import numpy as np
import pandas as pd
import os
import optuna
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgbm
from lightgbm import LGBMClassifier
import xgboost as xgb
from xgboost import XGBClassifier
import catboost
from catboost import CatBoostClassifier, Pool
from optuna.integration import LightGBMPruningCallback, XGBoostPruningCallback
import warnings
warnings.filterwarnings("ignore")

# Чтение данных

In [5]:
train = pd.read_csv('data/train.csv')
train.head()

,map_id,team1_id,team2_id,map_name,who_win
0,289,6665,7718,Ancient,0
1,715,4411,10577,Inferno,0
2,157,11251,9455,Nuke,1
3,524,4608,7532,Mirage,0
4,404,8637,6667,Overpass,1


In [6]:
players_feats = pd.read_csv('data/players_feats.csv')
players_feats.head()

,p1_id,p1_total_kills,p1_headshots,p1_total_deaths,p1_kd_ratio,p1_damage_per_round,p1_grenade_damage_per_round,p1_maps_played,p1_rounds_played,p1_kills_per_round,...,p5_kill_death_difference,p5_total_opening_kills,p5_total_opening_deaths,p5_opening_kill_ratio,p5_opening_kill_rating,p5_team_win_percent_after_first_kill,p5_first_kill_in_won_rounds,team_id,map_name,map_id
0,4954,90,42.2,112,0.80,76.3,5.9,6,156,0.58,...,5,25,12,2.08,1.28,84.0,25.0,6665,Ancient,635
1,5794,45,60.0,57,0.79,82.3,10.9,3,68,0.66,...,96,54,34,1.59,1.17,70.4,16.7,7532,Ancient,635
2,4954,156,51.9,167,0.93,63.5,3.4,10,265,0.59,...,22,26,19,1.37,1.10,88.5,20.5,6665,Dust2,583
3,5794,449,53.5,427,1.05,86.7,13.1,23,618,0.73,...,104,62,49,1.27,1.10,79.0,17.4,7532,Dust2,583
4,7998,173,32.9,130,1.33,82.4,2.9,9,225,0.77,...,19,27,25,1.08,1.08,81.5,16.2,4608,Dust2,439


In [7]:
#Фичи команды 1
team1_feats = train.merge(players_feats, how='left', left_on=['team1_id', 'map_id'], right_on=['team_id', 'map_id'])\
    .drop(['p1_id', 'p2_id', 'p3_id', 'p4_id', 'p5_id', 'team_id', 'map_name_y'], axis=1)
team1_feats.head()

,map_id,team1_id,team2_id,map_name_x,who_win,p1_total_kills,p1_headshots,p1_total_deaths,p1_kd_ratio,p1_damage_per_round,...,p5_kill_death,p5_kill_round,p5_rounds_with_kills,p5_kill_death_difference,p5_total_opening_kills,p5_total_opening_deaths,p5_opening_kill_ratio,p5_opening_kill_rating,p5_team_win_percent_after_first_kill,p5_first_kill_in_won_rounds
0,289,6665,7718,Ancient,0,258,36.0,293,0.88,71.1,...,1.21,0.74,210,55,55,33,1.67,1.18,83.6,19.7
1,715,4411,10577,Inferno,0,178,39.3,208,0.86,64.1,...,0.92,0.61,186,-23,47,49,0.96,1.01,74.5,16.3
2,157,11251,9455,Nuke,1,494,52.8,397,1.24,94.0,...,1.04,0.64,150,8,11,21,0.52,0.73,81.8,5.0
3,524,4608,7532,Mirage,0,474,29.1,304,1.56,86.9,...,1.20,0.75,261,66,74,57,1.30,1.18,81.1,20.3
4,404,8637,6667,Overpass,1,217,55.8,248,0.88,64.8,...,1.10,0.73,78,11,20,17,1.18,1.09,80.0,17.0


In [8]:
#Фичи команды 2
team2_feats = train.merge(players_feats, how='left', left_on=['team2_id', 'map_id'], right_on=['team_id', 'map_id'])\
    .drop(['p1_id', 'p2_id', 'p3_id', 'p4_id', 'p5_id', 'team_id', 'map_name_y'], axis=1)
team2_feats.head()

,map_id,team1_id,team2_id,map_name_x,who_win,p1_total_kills,p1_headshots,p1_total_deaths,p1_kd_ratio,p1_damage_per_round,...,p5_kill_death,p5_kill_round,p5_rounds_with_kills,p5_kill_death_difference,p5_total_opening_kills,p5_total_opening_deaths,p5_opening_kill_ratio,p5_opening_kill_rating,p5_team_win_percent_after_first_kill,p5_first_kill_in_won_rounds
0,289,6665,7718,Ancient,0,126,46.0,117,1.08,71.7,...,0.75,0.55,75,-34,21,20,1.05,1.06,76.2,19.0
1,715,4411,10577,Inferno,0,523,53.0,525,1.00,67.2,...,1.43,0.78,197,90,64,25,2.56,1.39,75.0,23.3
2,157,11251,9455,Nuke,1,201,39.8,215,0.93,69.0,...,0.81,0.55,101,-32,32,35,0.91,1.09,53.1,15.5
3,524,4608,7532,Mirage,0,366,53.6,359,1.02,85.4,...,0.92,0.54,109,-13,15,21,0.71,0.82,73.3,7.9
4,404,8637,6667,Overpass,1,184,50.0,189,0.97,73.9,...,1.17,0.72,122,26,20,10,2.00,1.01,80.0,12.4


In [9]:
#Дельта фичей
teams_feats = team1_feats.iloc[:,5:-1] - team2_feats.iloc[:,5:-1]
teams_feats

,p1_total_kills,p1_headshots,p1_total_deaths,p1_kd_ratio,p1_damage_per_round,p1_grenade_damage_per_round,p1_maps_played,p1_rounds_played,p1_kills_per_round,p1_assists_per_round,...,p5_rating,p5_kill_death,p5_kill_round,p5_rounds_with_kills,p5_kill_death_difference,p5_total_opening_kills,p5_total_opening_deaths,p5_opening_kill_ratio,p5_opening_kill_rating,p5_team_win_percent_after_first_kill
0,132,-10.0,176,-0.20,-0.6,1.5,10,271,-0.12,0.07,...,0.31,0.46,0.19,135,89,34,13,0.62,0.12,7.4
1,-345,-13.7,-317,-0.14,-3.1,3.7,-20,-516,-0.07,0.05,...,-0.23,-0.51,-0.17,-11,-113,-17,24,-1.60,-0.38,-0.5
2,293,13.0,182,0.31,25.0,1.3,9,259,0.23,0.02,...,-0.08,0.23,0.09,49,40,-21,-14,-0.39,-0.36,28.7
3,108,-24.5,-55,0.54,1.5,-4.1,1,33,0.16,-0.05,...,0.25,0.28,0.21,152,79,59,36,0.59,0.36,7.8
4,33,5.8,59,-0.09,-9.1,1.2,4,115,-0.12,0.03,...,-0.03,-0.07,0.01,-44,-15,0,7,-0.82,0.08,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,71,-5.2,134,-0.14,1.4,2.8,4,147,-0.03,-0.01,...,0.07,0.25,0.06,146,98,10,-6,0.75,-0.12,11.8
709,136,-9.5,54,0.68,24.5,1.9,5,122,0.29,-0.04,...,-0.29,0.84,0.69,27,-7,10,9,1.11,1.30,60.0
710,190,-5.1,232,-0.01,-6.1,2.2,14,385,-0.05,-0.01,...,0.26,0.28,0.16,189,107,53,30,0.43,0.19,1.7
711,236,-1.4,159,0.22,6.8,-2.2,10,241,0.14,-0.03,...,0.09,-0.03,-0.01,-121,-32,-12,-17,0.14,0.09,-6.9


In [10]:
#Получаем датафрем команда-карта
train_feats = pd.concat([train, teams_feats], axis=1)
train_feats.head()

,map_id,team1_id,team2_id,map_name,who_win,p1_total_kills,p1_headshots,p1_total_deaths,p1_kd_ratio,p1_damage_per_round,...,p5_rating,p5_kill_death,p5_kill_round,p5_rounds_with_kills,p5_kill_death_difference,p5_total_opening_kills,p5_total_opening_deaths,p5_opening_kill_ratio,p5_opening_kill_rating,p5_team_win_percent_after_first_kill
0,289,6665,7718,Ancient,0,132,-10.0,176,-0.20,-0.6,...,0.31,0.46,0.19,135,89,34,13,0.62,0.12,7.4
1,715,4411,10577,Inferno,0,-345,-13.7,-317,-0.14,-3.1,...,-0.23,-0.51,-0.17,-11,-113,-17,24,-1.60,-0.38,-0.5
2,157,11251,9455,Nuke,1,293,13.0,182,0.31,25.0,...,-0.08,0.23,0.09,49,40,-21,-14,-0.39,-0.36,28.7
3,524,4608,7532,Mirage,0,108,-24.5,-55,0.54,1.5,...,0.25,0.28,0.21,152,79,59,36,0.59,0.36,7.8
4,404,8637,6667,Overpass,1,33,5.8,59,-0.09,-9.1,...,-0.03,-0.07,0.01,-44,-15,0,7,-0.82,0.08,0.0


In [11]:
#Проверяем пустые значения
missing_data = train_feats.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print(missing_data[column].value_counts())
    print(" ")

map_id
False    713
Name: map_id, dtype: int64
 
team1_id
False    713
Name: team1_id, dtype: int64
 
team2_id
False    713
Name: team2_id, dtype: int64
 
map_name
False    713
Name: map_name, dtype: int64
 
who_win
False    713
Name: who_win, dtype: int64
 
p1_total_kills
False    713
Name: p1_total_kills, dtype: int64
 
p1_headshots
False    713
Name: p1_headshots, dtype: int64
 
p1_total_deaths
False    713
Name: p1_total_deaths, dtype: int64
 
p1_kd_ratio
False    713
Name: p1_kd_ratio, dtype: int64
 
p1_damage_per_round
False    713
Name: p1_damage_per_round, dtype: int64
 
p1_grenade_damage_per_round
False    713
Name: p1_grenade_damage_per_round, dtype: int64
 
p1_maps_played
False    713
Name: p1_maps_played, dtype: int64
 
p1_rounds_played
False    713
Name: p1_rounds_played, dtype: int64
 
p1_kills_per_round
False    713
Name: p1_kills_per_round, dtype: int64
 
p1_assists_per_round
False    713
Name: p1_assists_per_round, dtype: int64
 
p1_deaths_per_round
False    713
Name: 

In [12]:
train_feats = train_feats.fillna(0)

In [13]:
#Сохраняем получившийся датасет
train_feats.to_csv('train_feats.csv', index=False)

Подготавливаем тестовый датафрейм

In [14]:
test = pd.read_csv('data/test.csv').drop(['index'], axis=1)

In [15]:
#Тестовые фичи команды 1
team1_test_feats = test.merge(players_feats, how='left', left_on=['team1_id', 'map_id'], right_on=['team_id', 'map_id'])\
    .drop(['p1_id', 'p2_id', 'p3_id', 'p4_id', 'p5_id', 'team_id', 'map_name_y'], axis=1)
team1_test_feats

,map_id,team1_id,team2_id,map_name_x,p1_total_kills,p1_headshots,p1_total_deaths,p1_kd_ratio,p1_damage_per_round,p1_grenade_damage_per_round,...,p5_kill_death,p5_kill_round,p5_rounds_with_kills,p5_kill_death_difference,p5_total_opening_kills,p5_total_opening_deaths,p5_opening_kill_ratio,p5_opening_kill_rating,p5_team_win_percent_after_first_kill,p5_first_kill_in_won_rounds
0,309,5973,5752,Dust2,105,53.3,108,0.97,70.7,6.3,...,1.24,0.81,170,51,51,37,1.38,1.27,76.5,22.5
1,541,5973,5752,Vertigo,230,42.2,243,0.95,68.5,6.4,...,0.87,0.66,80,-18,24,33,0.73,1.02,50.0,14.3
2,1,8297,7020,Nuke,270,42.6,334,0.81,78.9,4.5,...,0.80,0.53,146,-50,26,33,0.79,0.86,76.9,11.8
3,392,8297,7020,Mirage,299,40.1,318,0.94,78.4,4.3,...,0.90,0.56,183,-27,28,32,0.88,0.86,75.0,9.5
4,684,8297,7020,Overpass,127,42.5,139,0.91,74.9,2.7,...,1.01,0.61,79,1,14,21,0.67,0.85,78.6,10.3
5,85,4494,4411,Vertigo,247,42.1,270,0.91,78.0,6.8,...,0.92,0.57,97,-11,16,19,0.84,0.87,87.5,11.4
6,314,4494,4411,Inferno,300,52.0,358,0.84,68.7,6.6,...,1.06,0.64,169,13,25,14,1.79,0.93,68.0,9.4
7,403,4494,4411,Mirage,242,51.2,322,0.75,62.9,4.0,...,1.04,0.66,176,9,35,38,0.92,0.95,82.9,14.8
8,125,4608,7718,Inferno,298,46.6,221,1.35,88.4,4.9,...,1.12,0.67,151,25,30,36,0.83,0.93,70.0,11.2
9,241,4608,7718,Mirage,525,32.2,340,1.54,87.1,2.0,...,1.16,0.73,281,60,81,62,1.31,1.18,75.3,18.4


In [16]:
#Тестовые фичи команды 2
team2_test_feats = test.merge(players_feats, how='left', left_on=['team2_id', 'map_id'], right_on=['team_id', 'map_id'])\
    .drop(['p1_id', 'p2_id', 'p3_id', 'p4_id', 'p5_id', 'team_id', 'map_name_y'], axis=1)
team2_test_feats

,map_id,team1_id,team2_id,map_name_x,p1_total_kills,p1_headshots,p1_total_deaths,p1_kd_ratio,p1_damage_per_round,p1_grenade_damage_per_round,...,p5_kill_death,p5_kill_round,p5_rounds_with_kills,p5_kill_death_difference,p5_total_opening_kills,p5_total_opening_deaths,p5_opening_kill_ratio,p5_opening_kill_rating,p5_team_win_percent_after_first_kill,p5_first_kill_in_won_rounds
0,309,5973,5752,Dust2,219,52.5,184,1.19,82.2,7.4,...,0.97,0.67,142,-5,44,33,1.33,1.24,70.5,18.9
1,541,5973,5752,Vertigo,198,40.4,184,1.08,76.5,7.6,...,0.89,0.62,125,-21,44,50,0.88,1.14,77.3,21.0
2,1,8297,7020,Nuke,231,51.9,208,1.11,70.6,4.0,...,1.33,0.83,17,6,3,2,1.50,1.07,66.7,15.4
3,392,8297,7020,Mirage,244,58.2,260,0.94,71.1,2.1,...,0.00,0.00,0,0,0,0,0.00,0.00,0.0,0.0
4,684,8297,7020,Overpass,197,55.8,182,1.08,71.6,3.3,...,1.43,0.80,13,6,4,2,2.00,1.33,100.0,25.0
5,85,4494,4411,Vertigo,182,31.3,212,0.86,61.3,8.0,...,0.94,0.60,141,-13,26,27,0.96,0.92,80.8,12.7
6,314,4494,4411,Inferno,209,29.2,209,1.00,60.6,4.4,...,0.98,0.57,141,-4,21,29,0.72,0.83,85.7,10.2
7,403,4494,4411,Mirage,202,29.2,191,1.06,67.1,4.6,...,1.01,0.61,136,1,13,19,0.68,0.78,76.9,6.6
8,125,4608,7718,Inferno,284,53.9,284,1.00,77.4,7.7,...,0.78,0.51,136,-55,28,23,1.22,0.93,82.1,11.5
9,241,4608,7718,Mirage,273,47.3,265,1.03,75.6,4.5,...,0.95,0.59,124,-10,24,39,0.62,0.84,91.7,13.5


In [18]:
#Дельта тестовых фичей
teams_test_feats = team1_test_feats.iloc[:,4:-1] - team2_test_feats.iloc[:,4:-1]
teams_test_feats

,p1_total_kills,p1_headshots,p1_total_deaths,p1_kd_ratio,p1_damage_per_round,p1_grenade_damage_per_round,p1_maps_played,p1_rounds_played,p1_kills_per_round,p1_assists_per_round,...,p5_rating,p5_kill_death,p5_kill_round,p5_rounds_with_kills,p5_kill_death_difference,p5_total_opening_kills,p5_total_opening_deaths,p5_opening_kill_ratio,p5_opening_kill_rating,p5_team_win_percent_after_first_kill
0,-114,0.8,-76,-0.22,-11.5,-1.1,-5,-124,-0.12,-0.02,...,0.18,0.27,0.14,28,56,7,4,0.05,0.03,6.0
1,32,1.8,59,-0.13,-8.0,-1.2,5,84,-0.07,-0.01,...,-0.05,-0.02,0.04,-45,3,-20,-17,-0.15,-0.12,-27.3
2,39,-9.3,126,-0.30,8.3,0.5,4,88,-0.05,0.07,...,-0.37,-0.53,-0.30,129,-56,23,31,-0.71,-0.21,10.2
3,55,-18.1,58,0.00,7.3,2.2,2,58,0.04,0.02,...,-0.45,0.90,0.56,183,-27,28,32,0.88,0.86,75.0
4,-70,-13.3,-43,-0.17,3.3,-0.6,-4,-110,0.02,0.05,...,0.01,-0.42,-0.19,66,-5,10,19,-1.33,-0.48,-21.4
5,65,10.8,58,0.05,16.7,-1.2,1,47,0.10,0.10,...,0.07,-0.02,-0.03,-44,2,-10,-8,-0.12,-0.05,6.7
6,91,22.8,149,-0.16,8.1,2.2,5,151,0.00,0.04,...,0.09,0.08,0.07,28,17,4,-15,1.07,0.10,-17.7
7,40,22.0,131,-0.31,-4.2,-0.6,4,111,-0.07,-0.01,...,0.11,0.03,0.05,40,8,22,19,0.24,0.17,6.0
8,14,-7.3,-63,0.35,11.0,-2.8,-3,-84,0.20,-0.04,...,0.18,0.34,0.16,15,80,2,13,-0.39,0.00,-12.1
9,252,-15.1,75,0.51,11.5,-2.5,5,182,0.22,-0.04,...,0.21,0.21,0.14,157,70,57,23,0.69,0.34,-16.4


In [19]:
#Тестовый датасет команда-карта
test_feats = pd.concat([test, teams_test_feats], axis=1)
test_feats

,map_id,team1_id,team2_id,map_name,p1_total_kills,p1_headshots,p1_total_deaths,p1_kd_ratio,p1_damage_per_round,p1_grenade_damage_per_round,...,p5_rating,p5_kill_death,p5_kill_round,p5_rounds_with_kills,p5_kill_death_difference,p5_total_opening_kills,p5_total_opening_deaths,p5_opening_kill_ratio,p5_opening_kill_rating,p5_team_win_percent_after_first_kill
0,309,5973,5752,Dust2,-114,0.8,-76,-0.22,-11.5,-1.1,...,0.18,0.27,0.14,28,56,7,4,0.05,0.03,6.0
1,541,5973,5752,Vertigo,32,1.8,59,-0.13,-8.0,-1.2,...,-0.05,-0.02,0.04,-45,3,-20,-17,-0.15,-0.12,-27.3
2,1,8297,7020,Nuke,39,-9.3,126,-0.30,8.3,0.5,...,-0.37,-0.53,-0.30,129,-56,23,31,-0.71,-0.21,10.2
3,392,8297,7020,Mirage,55,-18.1,58,0.00,7.3,2.2,...,-0.45,0.90,0.56,183,-27,28,32,0.88,0.86,75.0
4,684,8297,7020,Overpass,-70,-13.3,-43,-0.17,3.3,-0.6,...,0.01,-0.42,-0.19,66,-5,10,19,-1.33,-0.48,-21.4
5,85,4494,4411,Vertigo,65,10.8,58,0.05,16.7,-1.2,...,0.07,-0.02,-0.03,-44,2,-10,-8,-0.12,-0.05,6.7
6,314,4494,4411,Inferno,91,22.8,149,-0.16,8.1,2.2,...,0.09,0.08,0.07,28,17,4,-15,1.07,0.10,-17.7
7,403,4494,4411,Mirage,40,22.0,131,-0.31,-4.2,-0.6,...,0.11,0.03,0.05,40,8,22,19,0.24,0.17,6.0
8,125,4608,7718,Inferno,14,-7.3,-63,0.35,11.0,-2.8,...,0.18,0.34,0.16,15,80,2,13,-0.39,0.00,-12.1
9,241,4608,7718,Mirage,252,-15.1,75,0.51,11.5,-2.5,...,0.21,0.21,0.14,157,70,57,23,0.69,0.34,-16.4


In [20]:
#Проверяем пустые значения
missing_data = test_feats.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print(missing_data[column].value_counts())
    print(" ")

map_id
False    30
Name: map_id, dtype: int64
 
team1_id
False    30
Name: team1_id, dtype: int64
 
team2_id
False    30
Name: team2_id, dtype: int64
 
map_name
False    30
Name: map_name, dtype: int64
 
p1_total_kills
False    30
Name: p1_total_kills, dtype: int64
 
p1_headshots
False    30
Name: p1_headshots, dtype: int64
 
p1_total_deaths
False    30
Name: p1_total_deaths, dtype: int64
 
p1_kd_ratio
False    30
Name: p1_kd_ratio, dtype: int64
 
p1_damage_per_round
False    30
Name: p1_damage_per_round, dtype: int64
 
p1_grenade_damage_per_round
False    30
Name: p1_grenade_damage_per_round, dtype: int64
 
p1_maps_played
False    30
Name: p1_maps_played, dtype: int64
 
p1_rounds_played
False    30
Name: p1_rounds_played, dtype: int64
 
p1_kills_per_round
False    30
Name: p1_kills_per_round, dtype: int64
 
p1_assists_per_round
False    30
Name: p1_assists_per_round, dtype: int64
 
p1_deaths_per_round
False    30
Name: p1_deaths_per_round, dtype: int64
 
p1_saved_by_teammate_per_round

In [21]:
#Сохраняем получившийся датасет
test_feats.to_csv('test_feats.csv', index=False)

# LGBM + XGB + CatBoost + Optuna

In [30]:
train = pd.read_csv('data/train_feats.csv')
test = pd.read_csv('data/test_feats.csv')#.drop()

In [23]:
train.head()

,map_id,team1_id,team2_id,map_name,who_win,p1_total_kills,p1_headshots,p1_total_deaths,p1_kd_ratio,p1_damage_per_round,...,p5_rating,p5_kill_death,p5_kill_round,p5_rounds_with_kills,p5_kill_death_difference,p5_total_opening_kills,p5_total_opening_deaths,p5_opening_kill_ratio,p5_opening_kill_rating,p5_team_win_percent_after_first_kill
0,289,6665,7718,Ancient,0,132,-10.0,176,-0.20,-0.6,...,0.31,0.46,0.19,135,89,34,13,0.62,0.12,7.4
1,715,4411,10577,Inferno,0,-345,-13.7,-317,-0.14,-3.1,...,-0.23,-0.51,-0.17,-11,-113,-17,24,-1.60,-0.38,-0.5
2,157,11251,9455,Nuke,1,293,13.0,182,0.31,25.0,...,-0.08,0.23,0.09,49,40,-21,-14,-0.39,-0.36,28.7
3,524,4608,7532,Mirage,0,108,-24.5,-55,0.54,1.5,...,0.25,0.28,0.21,152,79,59,36,0.59,0.36,7.8
4,404,8637,6667,Overpass,1,33,5.8,59,-0.09,-9.1,...,-0.03,-0.07,0.01,-44,-15,0,7,-0.82,0.08,0.0


In [24]:
drop_cols_train = [
    'who_win',
    'map_name',
]

In [25]:
train_x = train.drop(columns=drop_cols_train, axis=1)
train_y = train[['who_win']]

In [155]:
def Optuna(argument):
    N_TRIALS = 20
    N_SPLITS = 5

    def LGBMObjective(trial, train_x = train_x, train_y = train_y):
        param = {
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
            "num_leaves": trial.suggest_int("num_leaves", 8, 4096),
            "max_depth": trial.suggest_int("max_depth", 3, 12),
            "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000),
            "lambda_l1": trial.suggest_int("lambda_l1", 0, 100),
            "lambda_l2": trial.suggest_int("lambda_l2", 0, 100),
            "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
            "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 0.95),
            "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 0.95),
            "device_type": 'gpu',
            "n_estimators": 10000,
            "bagging_freq": 1,
            "metric": 'auc',
            "objective": 'binary'
        }
        param = {
            "device_type": trial.suggest_categorical("device_type", ['gpu']),
            "n_estimators": trial.suggest_categorical("n_estimators", [10000]),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
            "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
            "max_depth": trial.suggest_int("max_depth", 3, 12),
            "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
            "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
            "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
            "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
            "bagging_fraction": trial.suggest_float(
              "bagging_fraction", 0.2, 0.95, step=0.1
            ),
            "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
            "feature_fraction": trial.suggest_float(
              "feature_fraction", 0.2, 0.95, step=0.1
        ),
    }
        cv = StratifiedKFold(n_splits=5, shuffle=True)
        cv_scores = np.empty(5)
        for idx, (train_idx, test_idx) in enumerate(cv.split(train_x, train_y)):
            X_train, X_valid = train_x.iloc[train_idx], train_x.iloc[test_idx]
            y_train, y_valid = train_y.iloc[train_idx], train_y.iloc[test_idx]
            model = LGBMClassifier(**param)
            model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], eval_metric="roc_auc_score", early_stopping_rounds=100,
                callbacks=[LightGBMPruningCallback(trial, "auc")], verbose = False
            )
            preds = model.predict_proba(X_valid)
            cv_scores[idx] = roc_auc_score(y_valid, preds[:,1])
        return np.mean(cv_scores)

    def XGBObjective(trial, x_train = train_x, y_train = train_y): 
        param = {
            'max_depth': trial.suggest_int('max_depth', 6, 10), 
            'n_estimators': trial.suggest_int('n_estimators', 400, 4000, 400), 
            'eta': trial.suggest_float('eta', 0.007, 0.013), 
            'subsample': trial.suggest_discrete_uniform('subsample', 0.2, 0.9, 0.1),
            'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree', 0.2, 0.9, 0.1),
            'colsample_bylevel': trial.suggest_discrete_uniform('colsample_bylevel', 0.2, 0.9, 0.1),
            'min_child_weight': trial.suggest_loguniform('min_child_weight', 1e-4, 1e4), 
            'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-4, 1e4),
            'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-4, 1e4), 
            'gamma': trial.suggest_loguniform('gamma', 1e-4, 1e4),
            'predictor': "gpu_predictor",
            'eval_metric' : 'auc',
            'objective' : 'binary:logistic',
            'tree_method': 'gpu_hist',
        }
        cv = StratifiedKFold(n_splits=5, shuffle=True)
        cv_scores = np.empty(5)
        for idx, (train_idx, test_idx) in enumerate(cv.split(train_x, train_y)):
            X_train, X_valid = train_x.iloc[train_idx], train_x.iloc[test_idx]
            y_train, y_valid = train_y.iloc[train_idx], train_y.iloc[test_idx]
            model = XGBClassifier(**param)
            model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=100, 
                      callbacks=[XGBoostPruningCallback(trial, 'validation_0-auc')], verbose = 0)
            preds = model.predict_proba(X_valid)
            cv_scores[idx] = roc_auc_score(y_valid, preds[:,1])
        return np.mean(cv_scores)

    def CatBoostObjective(trial, x_train = train_x, y_train = train_y):
        param = {
            'iterations':trial.suggest_int("iterations", 1000, 20000),
            'objective': trial.suggest_categorical('objective', ['Logloss', 'CrossEntropy']),
            'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli', 'MVS']),
            'od_wait':trial.suggest_int('od_wait', 500, 2000),
            'learning_rate' : trial.suggest_uniform('learning_rate',0.02,1),
            'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
            'random_strength': trial.suggest_uniform('random_strength',10,50),
            'depth': trial.suggest_int('depth',1,15),
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
            'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),
            'verbose': False,
            'task_type' : 'GPU',
            'devices' : '0',
            'eval_metric':'AUC',
            'od_type': 'IncToDec',
            'od_pval': 1e-7, 
            'od_wait' : 100,
        }
        if param['bootstrap_type'] == 'Bayesian':
            param['bagging_temperature'] = trial.suggest_float('bagging_temperature', 0, 10)
        elif param['bootstrap_type'] == 'Bernoulli':
            param['subsample'] = trial.suggest_float('subsample', 0.1, 1)
        cv = StratifiedKFold(n_splits=5, shuffle=True)
        cv_scores = np.empty(5)
        for idx, (train_idx, test_idx) in enumerate(cv.split(train_x, train_y)):
            X_train, X_valid = train_x.iloc[train_idx], train_x.iloc[test_idx]
            y_train, y_valid = train_y.iloc[train_idx], train_y.iloc[test_idx]
            model = CatBoostClassifier(**param)
            #Notice: Optuna Callback not supported on CatBoost
            cat_features = list(range(0, 4))
            model.fit(X_train, y_train, cat_features=list(range(0, 4)), eval_set=[(X_valid, y_valid)])
            preds = model.predict_proba(X_valid)
            cv_scores[idx] = roc_auc_score(y_valid, preds[:,1])
        return np.mean(cv_scores)
    def ObjectiveSelector(argument):
        objective = {
            'LGBM': LGBMObjective,
            'XGB': XGBObjective,
            'CatBoost': CatBoostObjective            
        }
        return objective.get(argument, "Invalid Selection")
    def ModelSelector(argument, trial): #Switch case not usable here without crashing. 
        if(argument == 'LGBM'):
            return LGBMClassifier(**trial.params)
        elif(argument == 'XGB'):
            return XGBClassifier(**trial.params)
        elif(argument == 'CatBoost'):
            return CatBoostClassifier(**trial.params)
        return "Invalid Model"
    study = optuna.create_study(direction="maximize")
    study.optimize(ObjectiveSelector(argument), n_trials=N_TRIALS)
    print("Number of finished trials: {}".format(len(study.trials)))
    print("Best trial:")
    trial = study.best_trial
    print("  Value: {}".format(trial.value))
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
    model = ModelSelector(argument, trial)
    model.fit(train_x, train_y)
    predictions_optuna = model.predict_proba(test)
    return predictions_optuna

In [ ]:
argument = 'XGB'
predictions_optuna = Optuna(argument)



In [ ]:
argument = 'LGBM'
predictions_optuna = Optuna(argument)

In [ ]:
argument = 'CatBoost'
predictions_optuna = Optuna(argument)

# H2O AUTOML

In [56]:
!apt-get install default-jre
!java -version

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-jre is already the newest version (2:1.11-68ubuntu1~18.04.1).
default-jre set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.
openjdk version "11.0.17" 2022-10-18
OpenJDK Runtime Environment (build 11.0.17+8-post-Ubuntu-1ubuntu218.04)
OpenJDK 64-Bit Server VM (build 11.0.17+8-post-Ubuntu-1ubuntu218.04, mixed mode, sharing)


In [57]:
!pip install h2o

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 177.5 MB 41 kB/s 
  Created wheel for h2o: filename=h2o-3.38.0.3-py2.py3-none-any.whl size=177541401 sha256=74886f836dbb583319296d4e29201924f4711e7b692d0d97c3986aa779b505ee
  Stored in directory: /root/.cache/pip/wheels/16/f8/f4/69e1ff6a0d1cb61bdbc0d9888ee8437a1acf1eb1c6ffb8be20
Successfully built h2o


In [26]:
import h2o

In [27]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,4 hours 13 mins
H2O_cluster_timezone:,Europe/Moscow
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.3
H2O_cluster_version_age:,11 days
H2O_cluster_name:,H2O_from_python_kikikita_i5667g
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.816 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [28]:
from h2o.automl import H2OAutoML

In [ ]:
train_df = h2o.H2OFrame(train)

In [62]:
train_df

map_id,team1_id,team2_id,map_name,who_win,p1_total_kills,p1_headshots,p1_total_deaths,p1_kd_ratio,p1_damage_per_round,p1_grenade_damage_per_round,p1_maps_played,p1_rounds_played,p1_kills_per_round,p1_assists_per_round,p1_deaths_per_round,p1_saved_by_teammate_per_round,p1_saved_teammates_per_round,p1_rating,p1_kill_death,p1_kill_round,p1_rounds_with_kills,p1_kill_death_difference,p1_total_opening_kills,p1_total_opening_deaths,p1_opening_kill_ratio,p1_opening_kill_rating,p1_team_win_percent_after_first_kill,p1_first_kill_in_won_rounds,p2_total_kills,p2_headshots,p2_total_deaths,p2_kd_ratio,p2_damage_per_round,p2_grenade_damage_per_round,p2_maps_played,p2_rounds_played,p2_kills_per_round,p2_assists_per_round,p2_deaths_per_round,p2_saved_by_teammate_per_round,p2_saved_teammates_per_round,p2_rating,p2_kill_death,p2_kill_round,p2_rounds_with_kills,p2_kill_death_difference,p2_total_opening_kills,p2_total_opening_deaths,p2_opening_kill_ratio,p2_opening_kill_rating,p2_team_win_percent_after_first_kill,p2_first_kill_in_won_rounds,p3_total_kills,p3_headshots,p3_total_deaths,p3_kd_ratio,p3_damage_per_round,p3_grenade_damage_per_round,p3_maps_played,p3_rounds_played,p3_kills_per_round,p3_assists_per_round,p3_deaths_per_round,p3_saved_by_teammate_per_round,p3_saved_teammates_per_round,p3_rating,p3_kill_death,p3_kill_round,p3_rounds_with_kills,p3_kill_death_difference,p3_total_opening_kills,p3_total_opening_deaths,p3_opening_kill_ratio,p3_opening_kill_rating,p3_team_win_percent_after_first_kill,p3_first_kill_in_won_rounds,p4_total_kills,p4_headshots,p4_total_deaths,p4_kd_ratio,p4_damage_per_round,p4_grenade_damage_per_round,p4_maps_played,p4_rounds_played,p4_kills_per_round,p4_assists_per_round,p4_deaths_per_round,p4_saved_by_teammate_per_round,p4_saved_teammates_per_round,p4_rating,p4_kill_death,p4_kill_round,p4_rounds_with_kills,p4_kill_death_difference,p4_total_opening_kills,p4_total_opening_deaths,p4_opening_kill_ratio,p4_opening_kill_rating,p4_team_win_percent_after_first_kill,p4_first_kill_in_won_rounds,p5_total_kills,p5_headshots,p5_total_deaths,p5_kd_ratio,p5_damage_per_round,p5_grenade_damage_per_round,p5_maps_played,p5_rounds_played,p5_kills_per_round,p5_assists_per_round,p5_deaths_per_round,p5_saved_by_teammate_per_round,p5_saved_teammates_per_round,p5_rating,p5_kill_death,p5_kill_round,p5_rounds_with_kills,p5_kill_death_difference,p5_total_opening_kills,p5_total_opening_deaths,p5_opening_kill_ratio,p5_opening_kill_rating,p5_team_win_percent_after_first_kill
289,6665,7718,Ancient,0,132,-10,176,-0.2,-0.6,1.5,10,271,-0.12,0.07,0.01,0.08,-0.03,-0.04,-0.2,-0.12,102,-44,16,18,0.06,-0.03,15.7,-1.2,130,-12.7,167,-0.14,-16.2,-1,10,271,-0.15,0.01,-0.06,0,0.02,-0.13,-0.14,-0.15,93,-37,11,24,-0.24,-0.3,10.9,-11.3,255,4.3,173,0.39,22.1,1,9,248,0.28,-0.01,-0.01,0.05,-0.02,0.37,0.39,0.28,152,82,61,51,0.56,0.48,6.4,16.1,31,2.8,21,0.04,5.5,2.1,1,27,0.06,0.03,0.03,0.07,0.06,0.01,0.04,0.06,26,10,-6,5,-1.57,-0.23,11.1,-4.8,217,-10.7,128,0.46,15.8,3.1,9,248,0.19,-0.02,-0.13,-0.1,0.07,0.31,0.46,0.19,135,89,34,13,0.62,0.12,7.4
715,4411,10577,Inferno,0,-345,-13.7,-317,-0.14,-3.1,3.7,-20,-516,-0.07,0.05,0.02,-0.01,-0.01,-0.06,-0.07,-0.03,-111,-3,-27,-46,0.72,0.03,-7,-1.2,-193,-4.5,-246,0.08,6,-1.4,-15,-393,0.07,0.02,0.02,0,0.06,0.03,0.12,0.08,-29,50,-20,-38,0.22,0,-4.2,-1.5,-309,-21.9,-221,-0.19,-2.6,6.1,-15,-393,-0.06,0.04,0.07,-0.01,0,-0.14,-0.18,-0.06,-83,-64,9,28,-0.34,0.09,-11.5,5.6,-422,10.1,-278,-0.28,-7.5,-3.4,-19,-493,-0.12,-0.03,0.04,0.03,-0.04,-0.13,-0.24,-0.13,-149,-88,-15,5,-0.45,0.11,13.7,8.4,-190,14.8,-49,-0.43,-3.6,2.8,-7,-180,-0.13,0.05,0.11,0.03,0.04,-0.23,-0.51,-0.17,-11,-113,-17,24,-1.6,-0.38,-0.5
157,11251,9455,Nuke,1,293,13,182,0.31,25,1.3,9,259,0.23,0.02,0.03,0.03,0.03,0.35,0.21,0.16,92,45,60,19,0.97,0.57,7.8,17.9,201,-4.2,145,0.11,3,-1.1,9,259,0.05,-0.01,-0.01,-0.04,-0.03,0.05,0.13,0.05,89,33,14,5,0.32,0.02,14.9,0.7,99,12.7,168,-0.32,-19.7,-0.4,9,259,-0.21,-0.04,0,-0.01,-0.05,-0.26,-0.3,-0.22,48,-53,-2,9,-0.6,-0.24,-6.4,-9.5,221,2

In [63]:
train_df.describe()

Rows:713
Cols:124

,map_id,team1_id,team2_id,map_name,who_win,p1_total_kills,p1_headshots,p1_total_deaths,p1_kd_ratio,p1_damage_per_round,p1_grenade_damage_per_round,p1_maps_played,p1_rounds_played,p1_kills_per_round,p1_assists_per_round,p1_deaths_per_round,p1_saved_by_teammate_per_round,p1_saved_teammates_per_round,p1_rating,p1_kill_death,p1_kill_round,p1_rounds_with_kills,p1_kill_death_difference,p1_total_opening_kills,p1_total_opening_deaths,p1_opening_kill_ratio,p1_opening_kill_rating,p1_team_win_percent_after_first_kill,p1_first_kill_in_won_rounds,p2_total_kills,p2_headshots,p2_total_deaths,p2_kd_ratio,p2_damage_per_round,p2_grenade_damage_per_round,p2_maps_played,p2_rounds_played,p2_kills_per_round,p2_assists_per_round,p2_deaths_per_round,p2_saved_by_teammate_per_round,p2_saved_teammates_per_round,p2_rating,p2_kill_death,p2_kill_round,p2_rounds_with_kills,p2_kill_death_difference,p2_total_opening_kills,p2_total_opening_deaths,p2_opening_kill_ratio,p2_opening_kill_rating,p2_team_win_percent_after_first_kill,p2_first_kill_in_won_rounds,p3_total_kills,p3_headshots,p3_total_deaths,p3_kd_ratio,p3_damage_per_round,p3_grenade_damage_per_round,p3_maps_played,p3_rounds_played,p3_kills_per_round,p3_assists_per_round,p3_deaths_per_round,p3_saved_by_teammate_per_round,p3_saved_teammates_per_round,p3_rating,p3_kill_death,p3_kill_round,p3_rounds_with_kills,p3_kill_death_difference,p3_total_opening_kills,p3_total_opening_deaths,p3_opening_kill_ratio,p3_opening_kill_rating,p3_team_win_percent_after_first_kill,p3_first_kill_in_won_rounds,p4_total_kills,p4_headshots,p4_total_deaths,p4_kd_ratio,p4_damage_per_round,p4_grenade_damage_per_round,p4_maps_played,p4_rounds_played,p4_kills_per_round,p4_assists_per_round,p4_deaths_per_round,p4_saved_by_teammate_per_round,p4_saved_teammates_per_round,p4_rating,p4_kill_death,p4_kill_round,p4_rounds_with_kills,p4_kill_death_difference,p4_total_opening_kills,p4_total_opening_deaths,p4_opening_kill_ratio,p4_opening_kill_rating,p4_team_win_percent_after_first_kill,p4_first_kill_in_won_rounds,p5_total_kills,p5_headshots,p5_total_deaths,p5_kd_ratio,p5_damage_per_round,p5_grenade_damage_per_round,p5_maps_played,p5_rounds_played,p5_kills_per_round,p5_assists_per_round,p5_deaths_per_round,p5_saved_by_teammate_per_round,p5_saved_teammates_per_round,p5_rating,p5_kill_death,p5_kill_round,p5_rounds_with_kills,p5_kill_death_difference,p5_total_opening_kills,p5_total_opening_deaths,p5_opening_kill_ratio,p5_opening_kill_rating,p5_team_win_percent_after_first_kill
type,int,int,int,enum,int,int,real,int,real,real,real,int,int,real,real,real,real,real,real,real,real,int,int,int,int,real,real,real,real,int,real,int,real,real,real,int,int,real,real,real,real,real,real,real,real,int,int,int,int,real,real,real,real,int,real,int,real,real,real,int,int,real,real,real,real,real,real,real,real,int,int,int,int,real,real,real,real,int,real,int,real,real,real,int,int,real,real,real,real,real,real,real,real,int,int,int,int,real,real,real,real,int,real,int,real,real,real,int,int,real,real,real,real,real,real,real,real,int,int,int,int,real,real,real
mins,0.0,4411.0,4411.0,,0.0,-730.0,-51.6,-711.0,-2.78,-91.1,-13.3,-41.0,-1097.0,-0.97,-0.19,-0.74,-0.12,-0.14,-1.41,-2.8,-0.98,-372.0,-238.0,-108.0,-82.0,-19.14,-1.33,-100.0,-37.3,-698.0,-57.9,-660.0,-1.62,-84.5,-15.4,-36.0,-975.0,-0.77,-0.2199999999999999,-0.7,-0.16,-0.16,-1.17,-1.62,-0.85,-338.0,-150.0,-103.0,-106.0,-7.91,-1.36,-87.5,-25.5,-648.0,-60.5,-642.0,-1.64,-90.0,-9.6,-35.0,-947.0,-0.82,-0.15,-0.65,-0.11,-0.15,-1.33,-3.0,-0.9,-310.0,-140.0,-90.0,-93.0,-4.43,-1.32,-90.9,-23.8,-684.0,-55.0,-649.0,-1.72,-81.2,-13.8,-34.0,-921.0,-0.79,-0.17,-0.73,-0.17,-0.14,-1.31,-1.75,-1.05,-325.0,-167.0,-102.0,-114.0,-5.83,-1.83,-100.0,-31.2,-675.0,-54.1,-576.0,-1.17,-80.4,-16.7,-35.0,-913.0,-0.74,-0.15,-0.68,-0.13,-0.18,-1.15,-1.97,-0.95,-296.0,-156.0,-85.0,-76.0,-3.52,-1.5,-100.0
mean,370.92566619915823,7243.643758765781,7787.844319775595,,0.4586255259467041,9.214586255259468,-0.6809256661991582,12.0224403927068

In [64]:
train_df['who_win'] = train_df['who_win'].asfactor()
train_df['map_id'] = train_df['map_id'].asfactor()
train_df['team1_id'] = train_df['team1_id'].asfactor()
train_df['team2_id'] = train_df['team2_id'].asfactor()
train_df['map_name'] = train_df['map_name'].asfactor()

In [111]:
feats_train,feats_test,feats_valid = train_df.split_frame(ratios=[.7, .15], seed = 1234)

In [108]:
y = "who_win"
x = train_df.columns
x.remove(y)
x.remove('map_id')
x.remove('team1_id')
x.remove('team2_id')
x.remove('map_name')

In [119]:
aml = H2OAutoML(max_models = 20, seed = 10, exclude_algos = ["StackedEnsemble","DeepLearning"], nfolds=0)

In [120]:
aml.train(x = x, y = y, training_frame = feats_train, validation_frame=feats_valid)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_1_AutoML_18_20221205_53117


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    30                 30                          7171                   1            3            2             2             4             3.16667

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.002114816042690116
RMSE: 0.04598712909815219
LogLoss: 0.04579653186590943
Mean Per-Class Error: 0.0
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9061292748928211
       0    1    Error    Rate
-----  ---  ---  -------  -----------
0      273  0    0        (0.0/273.0)
1      0    240  0        (0.0/240.0)
Total  273  240  0        (0.0/513.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value    idx
---------------------------  -----------  -------  -----
max f1                       0.906129     1        139
max f2                       0.906129     1        139
max f0point5                 0.906129     1        139
max accuracy                 0.906129     1        139
max precision                0.966817     1        0
max recall                   0.906129     1        139
max specificity              0.966817     1        0
max absolute_mcc             0.906129     1        139
max min_per_class_accuracy   0.906129     1        139
max mean_per_class_accuracy  0.906129     1        139
max tns                      0.966817     273      0
max fns                      0.966817     219      0
max fps                      0.03052      273      372
max tps                      0.906129     240      139
max tnr                      0.966817     1        0
max fnr                      0.966817     0.9125   0
max fpr                      0.03052      1        372
max tpr                      0.906129     1        139

Gains/Lift Table: Avg response rate: 46.78 %, avg score: 46.87 %
group    cumulative_data_fraction    lower_threshold    lift    cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain     cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  -------  -----------------  --------------------
1        0.0409357                   0.966817           2.1375  2.1375             1                0.966817   1                           0.966817            0.0875          0.0875                     113.75   113.75             0.0875
2        0.0409357                   0.966804           0       2.1375             0                0          1                           0.966817            0               0.0875                     -100     113.75             0.0875
3        0.0721248                   0.96679            2.1375  2.1375             1                0.96679    1                           0.966805            0.0666667       0.154167                   113.75   113.75             0.154167
4        0.105263                    0.961912           2.1375  2.1375             1                0.9635     1                           0.965765            0.0708333       0.225                      113.75   113.75             0.225
5        0.150097                    0.95908            2.1375  2.1375             1                0.960871   1                           0.964303            0.0958333       0.320833                   113.75   113.75             0.320833
6        0.20078                   

In [ ]:
test_df = h2o.H2OFrame(test.drop(['map_name'],axis=1))

In [ ]:
test_df['map_id'] = test_df['map_id'].asfactor()
test_df['team1_id'] = test_df['team1_id'].asfactor()
test_df['team2_id'] = test_df['team2_id'].asfactor()

In [ ]:
feats_pred=aml.leader.predict(test_df)

In [ ]:
h2o.export_file(feats_pred, path = "/content/drive/MyDrive/CSGO/XRT_prediction.csv", force = True)

# BEST_MODELS PREDICTIONS

In [36]:
best_catboost_model_1 = CatBoostClassifier(
    iterations = 17520,
    objective = 'Logloss',
    bootstrap_type = 'Bernoulli', 
    od_wait = 1464, 
    learning_rate = 0.2445390054694282, 
    reg_lambda = 50.47668497187493, 
    random_strength = 26.843079065440428, 
    depth = 1, 
    min_data_in_leaf = 11, 
    leaf_estimation_iterations = 10, 
    subsample = 0.11944499790217658
)
best_catboost_model_1.fit(
    train_x, train_y,
    cat_features = list(range(0,3)),
    verbose=False,
)

In [ ]:
best_catboost_model_2 = CatBoostClassifier(
    random_seed=63,
    iterations=10484, 
    objective='Logloss', 
    bootstrap_type='Bernoulli', 
    od_wait=1253, 
    learning_rate=0.5769703684226246, 
    reg_lambda=49.71966280773916, 
    random_strength=40.96205179962597,
    depth=4,
    min_data_in_leaf=22,
    leaf_estimation_iterations=11,
    subsample=0.7018775670299524
)
best_catboost_model_2.fit(
    train_x, train_y,
    cat_features = list(range(0,3)),
    verbose=False,
)

In [39]:
X_test = test.drop(columns='map_name', axis=1)
test_pool = Pool(data=X_test, cat_features = list(range(0,3)))
catboost_predictions = best_catboost_model_1.predict(test_pool)
print('Predictoins:')
print(catboost_predictions)

Predictoins:
[[0.40640066 0.59359934]
 [0.20390769 0.79609231]
 [0.23679221 0.76320779]
 [0.5692118  0.4307882 ]
 [0.941622   0.058378  ]
 [0.81713956 0.18286044]
 [0.12119244 0.87880756]
 [0.97848872 0.02151128]
 [0.94453354 0.05546646]
 [0.79095166 0.20904834]
 [0.99848759 0.00151241]
 [0.00819903 0.99180097]
 [0.06845009 0.93154991]
 [0.36179824 0.63820176]
 [0.94591169 0.05408831]
 [0.39331476 0.60668524]
 [0.07130097 0.92869903]
 [0.59595352 0.40404648]
 [0.22815685 0.77184315]
 [0.86000841 0.13999159]
 [0.07635172 0.92364828]
 [0.92959972 0.07040028]
 [0.85361996 0.14638004]
 [0.45548624 0.54451376]
 [0.89852334 0.10147666]
 [0.91986391 0.08013609]
 [0.94385546 0.05614454]
 [0.01754078 0.98245922]
 [0.99830056 0.00169944]
 [0.27065124 0.72934876]]


In [40]:
f = open('cat_1_prediction.csv', 'w')
f.write('map_id,who_win\n')
for idx in range(len(catboost_predictions)):
    line = str(test['map_id'][idx]) + ',' + str(catboost_predictions[idx]) + '\n'
    f.write(line)
f.close()

Saving best models

In [ ]:
best_catboost_model_1.save_model('best_catboost_model_1.bin')
best_catboost_model_1.save_model('best_catboost_model_1.json', format='json')


In [ ]:
best_catboost_model_2.save_model('best_catboost_model_2.bin')
best_catboost_model_2.save_model('best_catboost_model_2.json', format='json')